In [3]:
import sys
import os
import torch
import random
import gc

import matplotlib.pyplot        as plt
import torchaudio.transforms    as T
import torch.optim              as optim
import pandas                   as pd
import numpy                    as np

from torchvision       import transforms

sys.path.append(os.path.abspath(os.path.join('..')))
from utilit_train_test import make_weights_for_balanced_classes

sys.path.append(os.path.abspath(os.path.join('..','..','..','iESPnet_SRC_main','utilities')))
from Generator         import SeizureDatasetLabelTimev2, permute_spec, smoothing_label
from Model             import iESPnet
from TrainEval         import train_model_dsf_iespnet_abl_early, test_model_dsf_iespnet_abl, get_performance_indices

sys.path.append(os.path.abspath(os.path.join('../../../..','03 Dynamic-Spatial-Filtering')))
from models            import DynamicSpatialFilter

In [4]:
SPE_DIR        = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/'
meta_data_file = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/METADATA/allfiles_metadata.csv'

df_meta        = pd.read_csv(meta_data_file)


In [5]:
patients = df_meta['rns_id'].unique().tolist()

In [6]:
save_path = 'dsf_iespnet_lopo_early/'

In [7]:
s=0
save_runs        = save_path + patients[s] + '/runs/'
save_models      = save_path + patients[s] + '/models/'
save_predictions = save_path + patients[s] + '/results/'
save_figs        = save_path + patients[s] + '/figs/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

if not os.path.exists(save_runs):
    os.makedirs(save_runs)

if not os.path.exists(save_models):
    os.makedirs(save_models)

if not os.path.exists(save_predictions):
    os.makedirs(save_predictions)

if not os.path.exists(save_figs):
    os.makedirs(save_figs)

In [8]:
print('Running training for subject ' + patients[s] + ' [s]: ' + str(s))

Running training for subject PIT-RNS1603 [s]: 0


In [9]:
# define train y test de df_meta
train_df = df_meta.copy()
test_df  = df_meta[df_meta['rns_id'] == patients[s]]
test_df.reset_index(drop=True, inplace=True)
train_df.drop(train_df[train_df['rns_id'] == patients[s]].index, inplace = True)

patients_lopo = train_df[train_df['rns_id'] != patients[s]]['rns_id'].unique()

In [10]:
patients[s]

'PIT-RNS1603'

In [11]:
#np.random.seed(1)
patients_validation = []

In [12]:
# define train y test de df_meta
train_df = df_meta.copy()
test_df  = df_meta[df_meta['rns_id'] == patients[s]]
test_df.reset_index(drop=True, inplace=True)
train_df.drop(train_df[train_df['rns_id'] == patients[s]].index, inplace = True)

In [13]:
patients_lopo = [p for p in patients if p not in patients_validation and p != patients[s]]

In [25]:
torch.manual_seed(0)
random.seed(0)

In [26]:
random_patient = np.random.choice(patients_lopo)
print(random_patient)

PIT-RNS6806


In [232]:
random_patient

np.str_('PIT-RNS1534')

In [182]:
random_patient

np.str_('PIT-RNS1534')

In [183]:
val_df = train_df[train_df['rns_id'] == random_patient].copy()

In [184]:
val_df

,rns_id,data,label,time
22584,PIT-RNS1534,PIT_RNS1534_20161220-1_E0,0,0.0
22585,PIT-RNS1534,PIT_RNS1534_20161220-1_E1,0,0.0
22586,PIT-RNS1534,PIT_RNS1534_20161220-1_E2,0,0.0
22587,PIT-RNS1534,PIT_RNS1534_20161220-1_E3,0,0.0
22588,PIT-RNS1534,PIT_RNS1534_20161220-1_E4,0,0.0
...,...,...,...,...
23660,PIT-RNS1534,PIT_RNS1534_20171115-1_E165,0,0.0
23661,PIT-RNS1534,PIT_RNS1534_20171115-1_E166,0,0.0
23662,PIT-RNS1534,PIT_RNS1534_20171115-1_E167,0,0.0
23663,PIT-RNS1534,PIT_RNS1534_20171115-1_E168,0,0.0


In [160]:
train_df.drop(train_df[train_df['rns_id'] == random_patient].index, inplace=True)

In [161]:
patients_validation.append(random_patient)

In [162]:
# Resetear índices si es necesario
val_df.reset_index(drop=True, inplace=True)
train_df.reset_index(drop=True, inplace=True)

In [163]:
len(train_df.rns_id.unique())

28

In [165]:
train_df

,rns_id,data,label,time
0,PIT-RNS1603,PIT_RNS1603_20150804-1_E0,0,0.0
1,PIT-RNS1603,PIT_RNS1603_20150804-1_E1,0,0.0
2,PIT-RNS1603,PIT_RNS1603_20150804-1_E2,0,0.0
3,PIT-RNS1603,PIT_RNS1603_20150804-1_E3,0,0.0
4,PIT-RNS1603,PIT_RNS1603_20150804-1_E4,0,0.0
...,...,...,...,...
37121,PIT-RNS8076,PIT_RNS8076_20190806-1_E21,0,0.0
37122,PIT-RNS8076,PIT_RNS8076_20190806-1_E22,0,0.0
37123,PIT-RNS8076,PIT_RNS8076_20190806-1_E23,0,0.0
37124,PIT-RNS8076,PIT_RNS8076_20190806-1_E24,0,0.0


In [233]:
test_patient  = patients[s]
patients_lopo = [p for p in patients if p != test_patient]

In [237]:
patients_lopo

['PIT-RNS1603',
 'PIT-RNS6989',
 'PIT-RNS1529',
 'PIT-RNS7168',
 'PIT-RNS4098',
 'PIT-RNS1836',
 'PIT-RNS9183',
 'PIT-RNS1440',
 'PIT-RNS1713',
 'PIT-RNS9536',
 'PIT-RNS2543',
 'PIT-RNS1438',
 'PIT-RNS1534',
 'PIT-RNS1703',
 'PIT-RNS1556',
 'PIT-RNS1597',
 'PIT-RNS6992',
 'PIT-RNS8326',
 'PIT-RNS2227',
 'PIT-RNS8163',
 'PIT-RNS8973',
 'PIT-RNS6806',
 'PIT-RNS1090',
 'PIT-RNS2368',
 'PIT-RNS6762',
 'PIT-RNS2938',
 'PIT-RNS7525',
 'PIT-RNS0427',
 'PIT-RNS8076']

In [8]:
outputfile = save_models + 'model'
outputfile

'dsf_iespnet_lopo_early/PIT-RNS1603/models/model'

In [27]:
for s in range (0,7):
    print(s)

0
1
2
3
4
5
6
